In [39]:
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import re

In [6]:
!pip install kaggle

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle competitions download -c fake-news

fake-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [17]:
from zipfile import ZipFile
dataSet = '/content/fake-news.zip'

with ZipFile(dataSet,'r') as zip :
  zip.extractall()
  print("the dataset extracted")





the dataset extracted


In [23]:
import nltk
nltk.download('stopwords')

print(stopwords.words('english'))
print(stopwords.words('french'))
print(stopwords.words('arabic'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
news_dataSet = pd.read_csv('/content/train.csv')
news_dataSet



,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [33]:
#counitng the number of missing values in dataSet

news_dataSet.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [36]:
#remplace by empty string
news_dataSet = news_dataSet.fillna('')
news_dataSet.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [38]:
#merge title && autohr just cuz text is so large

news_dataSet['content'] = news_dataSet['title'] + ' ' + news_dataSet['author']

X = news_dataSet.drop('label',axis=1)
Y = news_dataSet['label']

In [54]:
#Stremming : is the process of reducing a word to its Root word

port_stem = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

news_dataSet['content']

0        hou dem aid even see comey letter jason chaffe...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exerci balkan alex a...
20799                            keep f aliv david swanson
Name: content, Length: 20800, dtype: object

In [52]:
news_dataSet['content'] = news_dataSet['content'].apply(stemming)

In [57]:
X = news_dataSet['content'].values
Y = news_dataSet['label'].values


In [60]:
X

array(['hou dem aid even see comey letter jason chaffetz tweet darrel lucu',
       'flynn hillari clinton big woman campu breitbart daniel j flynn',
       'truth might get fire consortiumnew com', ...,
       'maci said receiv takeov approach hudson bay new york time michael j de la merc rachel abram',
       'nato russia hold parallel exerci balkan alex ansari',
       'keep f aliv david swanson'], dtype=object)

In [59]:
Y

array([1, 0, 1, ..., 0, 1, 1])

In [74]:
#convert textual data to numerical data

vectorize = TfidfVectorizer()
vectorize.fit(X)

X_num_data = vectorize.transform(X)


X_train, X_test, Y_train, Y_test = train_test_split(X_num_data, Y, test_size = 0.2, stratify=Y, random_state=2)

In [76]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [78]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the training data :  0.9866586538461538
Accuracy score of the test data :  0.9790865384615385


In [79]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [80]:
print(Y_test[3])

0
